# DAT405/DIT407 Introduction to Data Science and AI 
## 2022-2023, Reading Period 4
## Assignment 4: Spam classification using Naïve Bayes 
The exercise takes place in this notebook environment.
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets. 
You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location: 
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
-	hard-ham: non-spam messages more difficult to differentiate 
-	spam: spam messages 

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.** If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer, with Windows you can use 
7zip (https://www.7-zip.org/download.html) to decompress the data.

**What to submit:** 
Convert the notebook to a pdf-file and submit it. Make sure all cells are executed so all your code and its results are included. Double check the pdf displays correctly before you submit it.

In [6]:
#Download and extract data
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2
!tar -xjf 20021010_easy_ham.tar.bz2
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2

--2023-04-21 14:19:18--  https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 151.101.2.132
Connecting to spamassassin.apache.org (spamassassin.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1677144 (1,6M) [application/x-bzip2]
Saving to: '20021010_easy_ham.tar.bz2'

     0K .......... .......... .......... .......... ..........  3% 10,1M 0s
    50K .......... .......... .......... .......... ..........  6% 2,45M 0s
   100K .......... .......... .......... .......... ..........  9% 3,29M 0s
   150K .......... .......... .......... .......... .......... 12% 2,55M 0s
   200K .......... .......... .......... .......... .......... 15% 2,54M 0s
   250K .......... .......... .......... .......... .......... 18% 2,87M 0s
   300K .......... .......... .......... .......... .......... 21% 2,06M 0s
   350K .......... .......... .......... .......... ..........

*The* data is now in the three folders `easy_ham`, `hard_ham`, and `spam`.

In [87]:
!ls -lah

'ls' is not recognized as an internal or external command,
operable program or batch file.


### 1. Preprocessing: 

##### 1.1 Look at a few emails from easy_ham, hard_ham and spam. Do you think you would be able to classify the emails just by inspection? How do you think a succesful model can learn the difference between the different classes of emails?


In [102]:
import os
easy_ham_files = os.listdir('./datasets/easy_ham/')
spam_files = os.listdir('./datasets/spam/')
easy_ham = []
spam = []
for file in easy_ham_files:
  f = open('./datasets/easy_ham/' + file,  encoding = "ISO-8859-1")
  easy_ham.append(f.read())
for file in spam_files:
  f = open('./datasets/spam/' + file,  encoding = "ISO-8859-1")
  spam.append(f.read())

print(easy_ham[1])
print(spam[1])


From Steve_Burt@cursor-system.com  Thu Aug 22 12:46:39 2002
Return-Path: <Steve_Burt@cursor-system.com>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id BE12E43C34
	for <zzzz@localhost>; Thu, 22 Aug 2002 07:46:38 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:46:38 +0100 (IST)
Received: from n20.grp.scd.yahoo.com (n20.grp.scd.yahoo.com
    [66.218.66.76]) by dogma.slashnull.org (8.11.6/8.11.6) with SMTP id
    g7MBkTZ05087 for <zzzz@example.com>; Thu, 22 Aug 2002 12:46:29 +0100
X-Egroups-Return: sentto-2242572-52726-1030016790-zzzz=example.com@returns.groups.yahoo.com
Received: from [66.218.67.196] by n20.grp.scd.yahoo.com with NNFMP;
    22 Aug 2002 11:46:30 -0000
X-Sender: steve.burt@cursor-system.com
X-Apparently-To: zzzzteana@yahoogroups.com
Received: (EGP: mail-8_1_0_1); 22 Aug 2002 11:46:

We think we would be able to classify the mails by looking at them but it would take a long time. We think a model could classify the mails successfully to some degree but some not with 100% accuracy

Answer 1.1:

##### 1.2 Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text (in the optional part further down can experiment with filtering out the headers and footers). We don’t want to train and test on the same data (it might help to reflect on why if you don't recall). Split the spam and the ham datasets in a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`). Use only the easy_ham part as ham data for quesions 1 and 2.

In [ ]:
# Write your code for here for splitting the data
from sklearn.model_selection import train_test_split

hamtrain, hamtest = train_test_split(easy_ham, test_size=0.3, random_state=0)
spamtrain, spamtest = train_test_split(spam, test_size=0.3, random_state=0)

### 2.1 Write a Python program that: 
1.	Uses the four datasets from Question 1 (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) 
2.	Trains a Naïve Bayes classifier (use the [scikit-learn library](https://scikit-learn.org/stable/)) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and False Negative rates on the `hamtest` and `spamtest` datasets. Use `CountVectorizer` ([Documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)) to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in scikit-learn ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Test two of these classifiers that are well suited for this problem:
- Multinomial Naive Bayes  
- Bernoulli Naive Bayes. 

Please inspect the documentation to ensure input to the classifiers is appropriate before you start coding. 



In [104]:
# Write your code here
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import confusion_matrix
#Create a Naïve Bayes Classifier

# vectorize the text
vect = CountVectorizer()
vect.fit(hamtrain)
hamtrain_dtm = vect.transform(hamtrain)
hamtest_dtm = vect.transform(hamtest)
spamtrain_dtm = vect.transform(spamtrain)
spamtest_dtm = vect.transform(spamtest)

ham_target = ['ham'] * len(hamtrain)
spam_target = ['spam'] * len(spamtrain)
mnb = MultinomialNB()
mnb.partial_fit(hamtrain_dtm, ham_target, classes=['ham', 'spam'])
mnb.partial_fit(spamtrain_dtm, spam_target)
bnb = BernoulliNB()
bnb.partial_fit(hamtrain_dtm, ham_target, classes=['ham', 'spam'])
bnb.partial_fit(spamtrain_dtm, spam_target)

print("Multinomial Naive Bayes On Easy Ham Versus Spam:")
mnb_ham_pred = mnb.predict(hamtest_dtm)
mnb_spam_pred = mnb.predict(spamtest_dtm)
ham_target = ['ham'] * len(hamtest)
spam_target = ['spam'] * len(spamtest)
mnb_ham_cm = confusion_matrix(ham_target, mnb_ham_pred)
mnb_spam_cm = confusion_matrix(spam_target, mnb_spam_pred)
mnb_TP = mnb_ham_cm[0][0]
mnb_FN = mnb_spam_cm[1][0]
mnb_TPR = mnb_TP/(mnb_TP + mnb_FN)
mnb_FNR = 1 - mnb_TPR
print("True Positive Rate:" + str(mnb_TPR))
print("False Negative Rate:" + str(mnb_FNR))
print("Bernoulli Naive Bayes On Easy Ham Versus Spam:")
bnb_ham_pred = bnb.predict(hamtest_dtm)
bnb_spam_pred = bnb.predict(spamtest_dtm)
bnb_ham_cm = confusion_matrix(ham_target, bnb_ham_pred)
bnb_spam_cm = confusion_matrix(spam_target, bnb_spam_pred)
bnb_TP = bnb_ham_cm[0][0]
bnb_FN = bnb_spam_cm[1][0]
bnb_TPR = bnb_TP/(bnb_TP + bnb_FN)
bnb_FNR = 1 - bnb_TPR
print("True Positive Rate:" + str(bnb_TPR))
print("False Negative Rate:" + str(bnb_FNR))



Multinomial Naive Bayes On Easy Ham Versus Spam:
True Positive Rate:0.9795134443021767
False Negative Rate:0.020486555697823317
Bernoulli Naive Bayes On Easy Ham Versus Spam:
True Positive Rate:0.9695044472681067
False Negative Rate:0.030495552731893305


### 2.2 Answer the following questions:
##### a) What does the CountVectorizer do?
Answer 2.2.a
##### b) What is the difference between Multinomial Naive Bayes and Bernoulli Naive Bayes
Answer 2.2.b


### 3.1 Run the two models:
Run (don't retrain) the two models from Question 2 on spam versus hard-ham. Does the performance differ compared to question 2 when the model was run on spam versus easy-ham? If so, why?

In [ ]:
# Write your code here

Answer 3.1:

### 3.2 Retrain
Retrain new Multinomial and Bernolli Naive Bayes classifers on the combined (easy+hard) ham and spam. Now evaluate on spam versus hard-ham as in 3.1. Also evaluate on spam versus easy-ham. Compare the performance with question 2 and 3.1. What do you observe?

In [ ]:
# Write your code here

Answer 3.2:

### 3.3 Further improvements
Do you have any suggestions for how performance could be further improved? You don't have to implement them, just present your ideas.

Answer 3.3: